In [1]:
import os
def setup_my_environment():
    import os
    
def setenv(var, val):
    os.environ[var] = val

 

def prepend_path(var, val):
    old_val = os.environ.get(var, '')
    os.environ[var] = val + ":" + old_val
def setup_java():
    PKG_ROOT='/ichec/packages/java/8'
    setenv('JAVA_PATH', PKG_ROOT)
    setenv('JAVA_HOME', PKG_ROOT)
    prepend_path('PATH', PKG_ROOT + '/bin')
    prepend_path('MANPATH', PKG_ROOT + '/man')
    prepend_path('CPATH', PKG_ROOT + '/include')
def setup_spark():
    PKG_ROOT='/ichec/packages/spark/2.3.3/kay/spark-2.3.3'
    setenv('SPARK_DIST_CLASSPATH', PKG_ROOT + 'spark-2.3.3-bin-kay-spark')
    prepend_path('PATH', PKG_ROOT + PKG_ROOT + 'spark-2.3.3-bin-kay-spark/bin')
setup_java()
setup_spark()
setup_my_environment()

In [2]:
from pyspark import SparkConf
from pyspark.context import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import lower, col
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import seaborn as sns
from pyspark.ml.classification import LogisticRegression, NaiveBayes, GBTClassifier
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel,LogisticRegressionWithSGD
from pyspark.sql.types import *
from nltk.corpus import stopwords
from nltk import pos_tag
import nltk

In [3]:
# spark = SparkSession \
#     .builder \
#     .config("spark.executor.memory", "70g") \
#     .config("spark.driver.memory", "90g") \
#     .config("spark.memory.offHeap.enabled","true")\
#     .config("spark.memory.offHeap.size","25g")\
#     .config("spark.debug.maxToStringsFields","100")\
#     .appName("AmazonSentimentAnalysis") \
#     .getOrCreate()

# spark = SparkSession \
#     .builder \
#     .config("spark.executor.memory", "6g") \
#     .config("spark.executor.instances","9")\
#     .config("spark.executor.cores","3")\
#     .config("spark.driver.memory", "2g") \
#     .config("spark.debug.maxToStringsFields","100")\
#     .appName("AmazonSentimentAnalysis") \
#     .getOrCreate()


spark = SparkSession \
    .builder \
    .config("spark.executor.memory", "25g") \
    .config("spark.executor.instances","7")\
    .config("spark.executor.cores","5")\
    .config("spark.debug.maxToStringsFields","100")\
    .appName("AmazonSentimentAnalysis") \
    .getOrCreate()


In [4]:
from pyspark.ml.feature import Tokenizer
import pandas as pd
import numpy as np
from pyspark.sql.functions import concat, col, lit, udf
from pyspark.sql import functions as sf
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.feature import HashingTF, IDF
from pyspark.ml import Pipeline
from pyspark.ml.feature import  RegexTokenizer
import re, string
from pyspark.sql.types import *
from nltk.stem import WordNetLemmatizer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from nltk.corpus import  wordnet
from collections import Counter

In [5]:
df1 = spark.read.json("/ichec/work/mucom001c/Amazon/review/AMAZON_FASHION.json")

In [6]:
df = df1

In [7]:
df = df.drop('verified','reviewerID','asin','reviewTime','reviewerName','unixReviewTime','vote','style','image')
df = df.dropna()
df = df.withColumn('reviewsummary', sf.concat(sf.col('reviewText'),sf.lit(' '), sf.col('summary')))

In [8]:
df =df.select("*", lower(col('reviewsummary')))
df = df.drop('reviewsummary')
df = df.withColumnRenamed("lower(reviewsummary)","reviewsummary")


In [9]:
df = df.drop('reviewText','summary')
# df.show(5)

In [10]:
df.groupBy("overall").count().orderBy(col("count").desc()).show()

+-------+------+
|overall| count|
+-------+------+
|    5.0|464263|
|    4.0|149103|
|    1.0|106916|
|    3.0| 96949|
|    2.0| 64669|
+-------+------+



In [11]:
def condition(rating):
    if (rating <3):
        label="negative"
    elif(rating >3):
        label="positive"
    else:
        label="neutral"
    return label
sentiment_udf = udf(lambda x: condition(x), StringType())

df = df.withColumn('Sentiment',sentiment_udf(df['overall']))


In [15]:
# df.show(10,True)

881900

In [12]:
def toBinary(score):
    if score >= 3: return 1
    else: return 0
udfScoretoBinary=udf(toBinary, StringType())

In [13]:
df = df.withColumn("Target", udfScoretoBinary("overall"))
# df.show(10)

In [14]:
def lower(text):
    return text.lower()

lower_udf=udf(lower,StringType())

In [15]:
##Remove nonAscii
def strip_non_ascii(data_str):
#''' Returns the string without non ASCII characters'''
    stripped = (c for c in data_str if 0 < ord(c) < 127)
    return ''.join(stripped)
# setup pyspark udf function
strip_non_ascii_udf = udf(strip_non_ascii, StringType())

In [22]:
##FIx abbreviations
def fix_abbreviation(data_str):
    data_str = data_str.lower()
    data_str = re.sub(r'\bthats\b', 'that is', data_str)
    data_str = re.sub(r'\bive\b', 'i have', data_str)
    data_str = re.sub(r'\bim\b', 'i am', data_str)
    data_str = re.sub(r'\bya\b', 'yeah', data_str)
    data_str = re.sub(r'\bcant\b', 'can not', data_str)
    data_str = re.sub(r'\bdont\b', 'do not', data_str)
    data_str = re.sub(r'\bwont\b', 'will not', data_str)
    data_str = re.sub(r'\bid\b', 'i would', data_str)
    data_str = re.sub(r'wtf', 'what the fuck', data_str)
    data_str = re.sub(r'\bwth\b', 'what the hell', data_str)
    data_str = re.sub(r'\br\b', 'are', data_str)
    data_str = re.sub(r'\bu\b', 'you', data_str)
    data_str = re.sub(r'\bk\b', 'OK', data_str)
    data_str = re.sub(r'\bsux\b', 'sucks', data_str)
    data_str = re.sub(r'\bno+\b', 'no', data_str)
    data_str = re.sub(r'\bcoo+\b', 'cool', data_str)
    data_str = re.sub(r'rt\b', '', data_str)
    data_str = data_str.strip()
    return data_str

In [16]:
##Remove punctuations mentions and alphanumeric characters

def remove_features(data_str):
# compile regex
    url_re = re.compile('https?://(www.)?\w+\.\w+(/\w+)*/?')
    punc_re = re.compile('[%s]' % re.escape(string.punctuation))
    num_re = re.compile('(\\d+)')
    mention_re = re.compile('@(\w+)')
    alpha_num_re = re.compile("^[a-z0-9_.]+$")
# convert to lowercase
    data_str = data_str.lower()
# remove hyperlinks
    data_str = url_re.sub(' ', data_str)
# remove @mentions
    data_str = mention_re.sub(' ', data_str)
# remove puncuation
    data_str = punc_re.sub(' ', str(data_str))
# remove numeric 'words'
    data_str = num_re.sub(' ', data_str)
# remove non a-z 0-9 characters and words shorter than 1 characters
    list_pos = 0
    cleaned_str = ''
    for word in data_str.split():
        if list_pos == 0:
            if alpha_num_re.match(word):
                cleaned_str = word
            else:
                cleaned_str = ' '
        else:
            if alpha_num_re.match(word):
                cleaned_str = cleaned_str + ' ' + word
            else:
                cleaned_str += ' '
        list_pos += 1
# remove unwanted space, *.split() will automatically split on
# whitespace and discard duplicates, the " ".join() joins the
# resulting list into one string.
    return " ".join(cleaned_str.split())
# setup pyspark udf function

In [17]:
##Remove stop words
def remove_stops(data_str):
# expects a string
    stopWords = set(stopwords.words("english"))
    list_pos = 0
    cleaned_str = ''
    text = data_str.split()
    for word in text:
        if word not in stopWords:
# rebuild cleaned_str
            if list_pos == 0:
                cleaned_str = word
            else:
                cleaned_str = cleaned_str + ' ' + word
            list_pos += 1
    return cleaned_str

In [18]:
# Part-of-Speech Tagging
def tag_and_remove(data_str):
    cleaned_str = ' '
# noun tags
    nn_tags = ['NN', 'NNP', 'NNP', 'NNPS', 'NNS']
# adjectives
    jj_tags = ['JJ', 'JJR', 'JJS']
# verbs
    vb_tags = ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']
    nltk_tags = nn_tags + jj_tags + vb_tags
# break string into 'words'
    text = data_str.split()
# tag the text and keep only those with the right tags
    tagged_text = pos_tag(text)
    for tagged_word in tagged_text:
        if tagged_word[1] in nltk_tags:
            cleaned_str += tagged_word[0] + ' '
    return cleaned_str

In [19]:
##Lemmatization
def lemmatize(data_str):
# expects a string
    list_pos = 0
    cleaned_str = ''
    lmtzr = WordNetLemmatizer()
    text = data_str.split()
    tagged_words = pos_tag(text)
    for word in tagged_words:
        if 'v' in word[1].lower():
            lemma = lmtzr.lemmatize(word[0], pos='v')
        else:
            lemma = lmtzr.lemmatize(word[0], pos='n')
        if list_pos == 0:
            cleaned_str = lemma
        else:
            cleaned_str = cleaned_str + ' ' + lemma
        list_pos += 1
    return cleaned_str

In [20]:
df = df.withColumn("lower_text",lower_udf(df["reviewsummary"]))

In [23]:
fix_abbreviation_udf = udf(fix_abbreviation, StringType())

In [24]:
remove_features_udf = udf(remove_features, StringType())

In [25]:

remove_stops_udf = udf(remove_stops, StringType())

In [26]:
tag_and_remove_udf = udf(tag_and_remove, StringType())

In [27]:
lemmatize_udf = udf(lemmatize, StringType())

In [34]:
nltk.download('stopwords')

[nltk_data] Error loading stopwords: <urlopen error [Errno 101]
[nltk_data]     Network is unreachable>


False

In [35]:
# df.show(10)

In [28]:
df = df.withColumn("text_non_asci",fix_abbreviation_udf(df["lower_text"]))
# df.show(10)

In [30]:
df = df.withColumn("fixed_abbrev",fix_abbreviation_udf(df["text_non_asci"]))
# df.show(10)


In [31]:
remove_features_udf = udf(remove_features, StringType())
df = df.withColumn('removed_features',remove_features_udf(df['fixed_abbrev']))
# df.show(10)

In [34]:
remove_stops_udf = udf(remove_stops, StringType())

In [35]:
df_no_stop_words = df.withColumn("removed_stops", remove_stops_udf("removed_features")).select('reviewsummary','removed_stops','Target')
# df_no_stop_words.show(5)
#if this line of code fails with pickling error run the above line "remove_stops_udf" and then run the current line.

In [95]:
df_no_stop_words.dtypes

[('reviewsummary', 'string'),
 ('removed_stops', 'string'),
 ('Target', 'string')]

In [36]:
df_pos_tagging=df_no_stop_words.withColumn("tag_and_remove_pos", tag_and_remove_udf("removed_stops")).select('reviewsummary','tag_and_remove_pos','Target')
# df_pos_tagging.show(5)

In [37]:
tokenizer = Tokenizer(inputCol="removed_stops", outputCol="words")
wordsDataFrame = tokenizer.transform(df_no_stop_words)
for words_label in wordsDataFrame.select("words", "Target").take(10):
    print(words_label)

Row(words=['exactly', 'needed', 'perfect', 'replacements'], Target='1')
Row(words=['agree', 'review', 'opening', 'small', 'almost', 'bent', 'hook', 'expensive', 'earrings', 'trying', 'get', 'higher', 'end', 'seen', 'would', 'buy', 'price', 'sending', 'back', 'agree', 'review', 'opening'], Target='0')
Row(words=['love', 'going', 'order', 'another', 'pack', 'keep', 'work', 'someone', 'including', 'always', 'losing', 'back', 'earring', 'understand', 'fish', 'hook', 'earrings', 'wish', 'tiny', 'bit', 'longer', 'new', 'friends'], Target='1')
Row(words=['tiny', 'opening', 'two', 'stars'], Target='0')
Row(words=['okay', 'three', 'stars'], Target='1')
Row(words=['exactly', 'wanted', 'five', 'stars'], Target='1')
Row(words=['little', 'plastic', 'backs', 'work', 'great', 'loosing', 'hook', 'earrings', 'wish', 'ordered', 'sooner', 'lost', 'favorite', 'earrings', 'works', 'great'], Target='1')
Row(words=['mother', 'law', 'wanted', 'present', 'sister', 'liked', 'said', 'would', 'work', 'bought', 'p

In [38]:
df_text = df.withColumn("text_lower",lower_udf(df["reviewsummary"])).select('text_lower','Target')

In [41]:
df_text.show(10)

+--------------------+------+
|          text_lower|Target|
+--------------------+------+
|exactly what i ne...|     1|
|i agree with the ...|     0|
|love these... i a...|     1|
|too tiny an openi...|     0|
|    okay three stars|     1|
|exactly what i wa...|     1|
|these little plas...|     1|
|mother - in - law...|     1|
|item is of good q...|     1|
|i had used my las...|     1|
+--------------------+------+
only showing top 10 rows



In [39]:
remover = StopWordsRemover(inputCol="words", outputCol="words_filtered")
wordsDataFrame1 = remover.transform(wordsDataFrame).select("Target","words_filtered")
# wordsDataFrame1.show(2)

In [40]:
df_text_lemma=df_no_stop_words.withColumn("lemmatized_text",lemmatize_udf("removed_stops")).select('reviewsummary','lemmatized_text','Target')
# df_text_lemma.show(100)

In [41]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [42]:
def getSentimentScore(reviewlemText):
    analyzer = SentimentIntensityAnalyzer()
    vs = analyzer.polarity_scores(reviewlemText)
    return float(vs['compound'])

def getCleanreviewText(filteredreviewText1):
    return ' '.join(filteredreviewText1)

In [43]:
udfCleanreviewText = udf(getCleanreviewText, StringType())
dfFilteredCleanedTweet = df_text_lemma.withColumn('filteredCleanedreviewText', udfCleanreviewText('lemmatized_text'))
dfFilteredCleanedTweet.select('Target','filteredCleanedreviewText')

DataFrame[Target: string, filteredCleanedreviewText: string]

In [44]:
udfSentimentScore = udf(getSentimentScore ) #,FloatType()
dfSentimentScore = dfFilteredCleanedTweet.withColumn('sentimentScore', udfSentimentScore('lemmatized_text'))
dfSentimentScore.select('Target','lemmatized_text','sentimentScore')

DataFrame[Target: string, lemmatized_text: string, sentimentScore: string]

In [45]:
def getSentimentScoreneg(reviewlemText):
    analyzer = SentimentIntensityAnalyzer()
    vs = analyzer.polarity_scores(reviewlemText)
    return float(vs['neg'])

In [46]:
udfSentimentScoreneg = udf(getSentimentScoreneg, FloatType())
dfSentimentScoreneg = dfSentimentScore.withColumn('sentimentneg', udfSentimentScoreneg('lemmatized_text'))
dfSentimentScoreneg.select('Target','lemmatized_text','sentimentScore','sentimentneg')

DataFrame[Target: string, lemmatized_text: string, sentimentScore: string, sentimentneg: float]

In [47]:
def getSentimentScoreneu(reviewlemText):
    analyzer = SentimentIntensityAnalyzer()
    vs = analyzer.polarity_scores(reviewlemText)
    return float(vs['neu'])

In [48]:
udfSentimentScoreneu = udf(getSentimentScoreneu, FloatType())
dfSentimentScoreneu = dfSentimentScoreneg.withColumn('sentimentneu',udfSentimentScoreneu('lemmatized_text'))
dfSentimentScoreneu.select('Target','lemmatized_text','sentimentScore','sentimentneg','sentimentneu')

DataFrame[Target: string, lemmatized_text: string, sentimentScore: string, sentimentneg: float, sentimentneu: float]

In [49]:
def getSentimentScorepos(reviewlemText):
    analyzer = SentimentIntensityAnalyzer()
    vs = analyzer.polarity_scores(reviewlemText)
    return float(vs['pos'])

In [50]:
udfSentimentScorepos = udf(getSentimentScorepos, FloatType())
dfSentimentScorepos = dfSentimentScoreneu.withColumn('sentimentpos', udfSentimentScorepos('lemmatized_text'))
dfSentimentScorepos.select('Target','filteredCleanedreviewText','sentimentScore','sentimentneu','sentimentneg','lemmatized_text')

DataFrame[Target: string, filteredCleanedreviewText: string, sentimentScore: string, sentimentneu: float, sentimentneg: float, lemmatized_text: string]

In [51]:
dfSentimentScorepos.select('Target','lemmatized_text','sentimentScore','sentimentneu','sentimentneg','sentimentpos')

DataFrame[Target: string, lemmatized_text: string, sentimentScore: string, sentimentneu: float, sentimentneg: float, sentimentpos: float]

In [52]:
dfmodel_data = dfSentimentScorepos
dfmodel_data

DataFrame[reviewsummary: string, lemmatized_text: string, Target: string, filteredCleanedreviewText: string, sentimentScore: string, sentimentneg: float, sentimentneu: float, sentimentpos: float]

In [53]:
dfmodel_data = dfmodel_data.drop('sentimentScore','lemmatized_text','reviewsummary','filteredCleanedreviewText')

In [54]:
cols=dfmodel_data.columns
cols.remove("Target")

+------+------------+------------+------------+
|Target|sentimentneg|sentimentneu|sentimentpos|
+------+------------+------------+------------+
|     1|         0.0|       0.448|       0.552|
|     0|         0.0|         0.8|         0.2|
|     1|       0.085|       0.597|       0.318|
|     0|         0.0|         1.0|         0.0|
|     1|         0.0|       0.513|       0.487|
+------+------------+------------+------------+
only showing top 5 rows



In [55]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=cols,outputCol="features")
df_model_data=assembler.transform(dfmodel_data)
df_model_data.select("features").show(truncate=False)

+-------------------------------------------------------------+
|features                                                     |
+-------------------------------------------------------------+
|[0.0,0.4480000138282776,0.5519999861717224]                  |
|[0.0,0.800000011920929,0.20000000298023224]                  |
|[0.08500000089406967,0.597000002861023,0.3179999887943268]   |
|[0.0,1.0,0.0]                                                |
|[0.0,0.5130000114440918,0.4869999885559082]                  |
|[0.0,0.6980000138282776,0.3019999861717224]                  |
|[0.09799999743700027,0.39899998903274536,0.5040000081062317] |
|[0.0,0.703000009059906,0.296999990940094]                    |
|[0.027000000700354576,0.6050000190734863,0.36800000071525574]|
|[0.05900000035762787,0.6179999709129333,0.3230000138282776]  |
|[0.13699999451637268,0.609000027179718,0.2540000081062317]   |
|[0.20499999821186066,0.7950000166893005,0.0]                 |
|[0.0,1.0,0.0]                          

In [56]:
from pyspark.sql.types import IntegerType
df_model_data = df_model_data.withColumn("Targetint",df_model_data["Target"].cast(IntegerType()))
df_model_data.dtypes

[('Target', 'string'),
 ('sentimentneg', 'float'),
 ('sentimentneu', 'float'),
 ('sentimentpos', 'float'),
 ('features', 'vector'),
 ('Targetint', 'int')]

In [57]:
df_model_data = df_model_data.drop('Target')

In [58]:
train, test = df_model_data.randomSplit([0.7, 0.3], seed=42)

In [ ]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(labelCol="Targetint", featuresCol="features",maxIter=5)
model=lr.fit(train)
predict_train=model.transform(train)
predict_test=model.transform(test)

In [ ]:
predict_test.select("Targetint","prediction").show(10)

In [86]:

lrevaluator = MulticlassClassificationEvaluator(labelCol = 'Targetint',predictionCol = 'prediction', metricName = 'accuracy')
lrevaluator.evaluate(predict_test)

0.8411256378879893

In [64]:
y_true = predict_test.select(['Targetint']).collect()
y_pred = predict_test.select(['prediction']).collect()

KeyboardInterrupt: 

In [ ]:
print(classification_report(y_true, y_pred))

In [ ]:

print(confusion_matrix(y_true, y_pred))

In [88]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.61      0.49      0.54    120366
           1       0.88      0.93      0.90    497577

    accuracy                           0.84    617943
   macro avg       0.75      0.71      0.72    617943
weighted avg       0.83      0.84      0.83    617943



In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_true, y_pred))

In [ ]:
from mlxtend.plotting import plot_confusion_matrix
import matplotlib.pyplot as plt
import numpy as np

binary1 = np.array([[0.61, 0.49],
                   [0.88, 0.93]])

fig, ax = plot_confusion_matrix(conf_mat=binary1)
plt.show()

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
lrevaluator = MulticlassClassificationEvaluator(labelCol = 'Targetint',predictionCol = 'prediction', metricName = 'f1')
lrevaluator.evaluate(predict_test)

In [59]:
nb = NaiveBayes(smoothing=1.0, modelType="multinomial",featuresCol="features", labelCol="Targetint")
model_nb = nb.fit(train)

predict_train_nb=model_nb.transform(train)
predict_test_nb=model_nb.transform(test)


In [ ]:
predict_test_nb.select("Targetint","prediction").show(10)

In [64]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
lrevaluator = MulticlassClassificationEvaluator(labelCol = 'Targetint',predictionCol = 'prediction', metricName = 'accuracy')
lrevaluator.evaluate(predict_test_nb)

0.806066897259781

In [65]:
y_true_nb = predict_train_nb.select(['Targetint']).collect()
y_pred_nb = predict_train_nb.select(['prediction']).collect()

In [66]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_true_nb, y_pred_nb))

              precision    recall  f1-score   support

           0       0.81      0.00      0.00    120366
           1       0.81      1.00      0.89    497577

    accuracy                           0.81    617943
   macro avg       0.81      0.50      0.45    617943
weighted avg       0.81      0.81      0.72    617943



In [76]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_true_nb, y_pred_nb))

[[    55 120311]
 [    13 497564]]


In [60]:
y_true_nb = predict_test_nb.select(['Targetint']).collect()
y_pred_nb = predict_test_nb.select(['prediction']).collect()

KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_true_nb, y_pred_nb))

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_true_nb, y_pred_nb))

In [61]:
from pyspark.mllib.evaluation import MulticlassMetrics
nb_test = predict_test_nb.select('prediction','Targetint').rdd
metrics = MulticlassMetrics (nb_test)
print(metrics.confusionMatrix())

Py4JJavaError: An error occurred while calling o780.confusionMatrix.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 9.0 failed 1 times, most recent failure: Lost task 0.0 in stage 9.0 (TID 403, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/ichec/home/users/pawan2592/.conda/envs/myenv/lib/python3.5/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/ichec/home/users/pawan2592/.conda/envs/myenv/lib/python3.5/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/ichec/home/users/pawan2592/.conda/envs/myenv/lib/python3.5/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 400, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/ichec/home/users/pawan2592/.conda/envs/myenv/lib/pytho

In [4]:
from mlxtend.plotting import plot_confusion_matrix
import matplotlib.pyplot as plt
import numpy as np

binary1 = np.array([[0.81, 0.00],
                   [0.81, 1.00]])

fig,ax = plot_confusion_matrix(conf_mat=binary1)
plt.show()

NameError: name 'np' is not defined

In [67]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
lrevaluator = MulticlassClassificationEvaluator(labelCol = 'Targetint',predictionCol = 'prediction', metricName = 'f1')
lrevaluator.evaluate(predict_test_nb)

0.7196234964040157

In [68]:
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(numTrees = 100,maxDepth = 4,maxBins = 32,featuresCol="features", labelCol="Targetint")
# Train model with Training Data
model_rf = rf.fit(train)
predict_train=model_rf.transform(train)
predict_test=model_rf.transform(test)
predict_test.select("Targetint","prediction").show(10)

+---------+----------+
|Targetint|prediction|
+---------+----------+
|        0|       1.0|
|        1|       1.0|
|        1|       1.0|
|        1|       1.0|
|        1|       1.0|
|        1|       1.0|
|        1|       1.0|
|        1|       1.0|
|        1|       1.0|
|        1|       1.0|
+---------+----------+
only showing top 10 rows



In [69]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
lrevaluator = MulticlassClassificationEvaluator(labelCol = 'Targetint',predictionCol = 'prediction', metricName = 'accuracy')
lrevaluator.evaluate(predict_test)

0.8422243016855018

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
lrevaluator = MulticlassClassificationEvaluator(labelCol = 'Targetint',predictionCol = 'prediction', metricName = 'f1')
lrevaluator.evaluate(predict_test)

In [72]:
y_true_rf = predict_train.select(['Targetint']).collect()
y_pred_rf = predict_train.select(['prediction']).collect()

In [74]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_true_rf, y_pred_rf))

[[ 35838  84528]
 [ 13298 484279]]


In [75]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_true_rf, y_pred_rf))

              precision    recall  f1-score   support

           0       0.73      0.30      0.42    120366
           1       0.85      0.97      0.91    497577

    accuracy                           0.84    617943
   macro avg       0.79      0.64      0.67    617943
weighted avg       0.83      0.84      0.81    617943



In [ ]:
from pyspark.ml.classification import GBTClassifier
## Fitting the model
gbt = GBTClassifier(maxIter=10,featuresCol="features", labelCol="Targetint")
predict_train_gbt = gbt.fit(train)
predict_train_gbt = gbtModel.transform(test)
## Evaluating the model


In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol="Targetint", predictionCol="prediction", metricName="accuracy")
gb_accuracy = evaluator.evaluate(predict_train_gbt)
print("Accuracy of GBT is = %g"% (gb_accuracy))